# Virtual experiments, replicating Lahiri and Bevan 2020

In this notebook, we try and replicate a few of the dopamine experiments in Lahiri and Bevan 2020.

4Hz optogenetical stimulation (5 pulses) yielded approximately 0.3 uM DA concentration, while 20Hz stimulation yielded (5 pulses) on average 0.5 uM DA concentration. Here we model the optogenetic activation by setting the DA concentration to these values. In both cases the duration of the resulting DA pulse was set to 5 seconds. 


Experiment 4¶

Subthreshold current injection (<10 mV depolarisation). Size of depolarisation not affected by DA.

Suprathreshold current injection, with DA lower threshold, and faster spiking.

Compare Figure 5

In [1]:
import os
from snudda import Snudda

neuron = "dspn"
# neuron = "ispn"

neuron_path = os.path.join("data", neuron)
network_path = os.path.join("networks", "bevan_fig4_bath_subthresh_current_SBML")

In [2]:
snudda = Snudda(network_path=network_path)
si = snudda.init_tiny(neuron_paths=neuron_path, neuron_names=neuron, number_of_neurons=[1], 
                      random_seed=1234)

si.network_data["regions"]["Cube"]["neurons"]["dspn"]["reaction_diffusion"] = "data/JSON/reaction_diffusion_D1.json"
# si.network_data["regions"]["Cube"]["neurons"]["ispn"]["reaction_diffusion"] = "data/JSON/reaction_diffusion_D2.json-updated"


# How the ion channels are modified by DA
# OBS, we include SK direkt modulation, in relality it should be modulated by DA acting on Ca 
# si.network_data["regions"]["Cube"]["neurons"][neuron]["modulation"] = "modulation_parameters.json"
si.network_data["regions"]["Cube"]["neurons"][neuron]["modulation"] = "data/modulation-bevan2020.json"


si.network_data["regions"]["Cube"]["neurons"][neuron]["modulation_key"] = "abc"

si.write_json()

snudda.create_network()

Warning, no snudda_data set!
Adding neurons: dspn from dir data/dspn
Writing networks/bevan_fig4_bath_subthresh_current_SBML/network-config.json
Writing networks/bevan_fig4_bath_subthresh_current_SBML/network-config.json
Placing neurons
Network path: networks/bevan_fig4_bath_subthresh_current_SBML
Creating missing directory networks/bevan_fig4_bath_subthresh_current_SBML/log
Created directory networks/bevan_fig4_bath_subthresh_current_SBML/log
Reading SNUDDA_DATA=None from networks/bevan_fig4_bath_subthresh_current_SBML/network-config.json
No n_putative_points and putative_density, setting n_putative_points = 46
(this must be larger than the number of neurons you want to place)
Generating 46 points for networks/bevan_fig4_bath_subthresh_current_SBML/mesh/Cube-cube-mesh-2.3159794767993218e-05.obj
Filtering, keeping inside points: 1 / 22
Managed to create 1 putative points within the volume.
  WARNING --> is the volume too small? You can create new cube mesh using create_cube_mesh.py

Ex

In [3]:
# Free memory
snudda = None

mech_dir = "/home/hjorth/HBP/BasalGangliaData/data/neurons/mechanisms"
sample_dt = None # 0.00005

sim_config_on = os.path.join("data", "bevan_fig4_with_DA_sbml.json")
sim_config_off = os.path.join("data", "bevan_fig4_no_DA_sbml.json")

sim_output_neuromodulation_ON = os.path.join(network_path, "simulation", "output_neuromodulation_ON.hdf5")
sim_output_neuromodulation_OFF = os.path.join(network_path, "simulation", "output_neuromodulation_OFF.hdf5")

sim_time = 2
n_workers = 1

In [4]:
run_str_on = f"snudda simulate {network_path} --time {sim_time} --simulation_config {sim_config_on} --mechdir {mech_dir} --enable_rxd_neuromodulation --verbose"
print(run_str_on)

snudda simulate networks/bevan_fig4_bath_subthresh_current_SBML --time 2 --simulation_config data/bevan_fig4_with_DA_sbml.json --mechdir /home/hjorth/HBP/BasalGangliaData/data/neurons/mechanisms --enable_rxd_neuromodulation --verbose


In [ ]:
os.system(run_str_on)

In [5]:
run_str_off = f"snudda simulate {network_path} --time {sim_time} --simulation_config {sim_config_off} --mechdir {mech_dir} --verbose"
print(run_str_off)

snudda simulate networks/bevan_fig4_bath_subthresh_current_SBML --time 2 --simulation_config data/bevan_fig4_no_DA_sbml.json --mechdir /home/hjorth/HBP/BasalGangliaData/data/neurons/mechanisms --verbose


In [ ]:
os.system(run_str_off)

In [6]:
from snudda.utils import SnuddaLoadSimulation

nd = SnuddaLoadSimulation(sim_output_neuromodulation_ON)
time = nd.get_time()
data_pka = nd.get_data("PKAc", 0)[0][0]
data_da = nd.get_data("DA", 0)[0][0]
data_da_external = nd.get_data("DA", 0)[0][0]

# This is saved with add_rxd_internal_concentration_recording_all -- check that it worked 
data_pka_all0 = nd.get_data("PKAc", 0)[0][0]

Loading networks/bevan_fig4_bath_subthresh_current_SBML/simulation/output_neuromodulation_ON.hdf5


In [7]:
nd.list_data_types(0)

['AC5',
 'AC5GaolfGTP',
 'AC5GaolfGTP_ATP',
 'AC5_ATP',
 'AMP',
 'ATP',
 'D1R',
 'D1RDA',
 'D1RDAGolf',
 'D1RGolf',
 'DA',
 'GaolfGDP',
 'GaolfGTP',
 'Gbgolf',
 'Golf',
 'PDE10',
 'PDE10_cAMP',
 'PDE10c',
 'PDE10c_cAMP',
 'PDE4',
 'PDE4_cAMP',
 'PKA',
 'PKAc',
 'PKAcAMP2',
 'PKAcAMP4',
 'PKAreg',
 'cAMP',
 'cal12_ms.modulation_factor',
 'cal13_ms.modulation_factor',
 'kaf_ms.modulation_factor_g',
 'kaf_ms.modulation_factor_shift',
 'kas_ms.modulation_factor',
 'naf_ms.modulation_factor',
 'sk_ms.modulation_factor',
 'spikes',
 'voltage']

In [8]:
data_types = nd.list_data_types(0)
all_species_data = nd.get_all_data(neuron_id=0, exclude=["spikes", "voltage"])
time = nd.get_time()
voltage = nd.get_data("voltage", [0])

In [9]:
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = "iframe"  # Do not save plots in the notebook, they can get BIG

fig = go.Figure()
for data_type in all_species_data:
    idx = time >= 0.0
    fig.add_trace(go.Scatter(x=time[idx], y=all_species_data[data_type][0][0].T[0][idx], name=data_type, line={"width":4}))

"""
fig.update_layout(xaxis_title="Time (s)", yaxis_title="Concentration", width=1000, height=800,
                 font={"size":18},  # General font size for all elements
                 legend={"font":{"size":16}},  # Specific font size for legend
                 xaxis={"title":{"font":{"size":20}}, "tickfont":{"size":14}},  # X-axis title and tick labels
                 yaxis={"title":{"font":{"size":20}}, "tickfont":{"size":14}})   # Y-axis title and tick labels
"""
fig.show()

In [12]:
# Reporting plot

import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = "iframe"  # Do not save plots in the notebook, they can get BIG

fig = go.Figure()
for data_type in ["D1RDA", "PKAc", "cAMP"]: #all_species_data:
    idx = time >= 0.0
    fig.add_trace(go.Scatter(x=time[idx], y=all_species_data[data_type][0][0].T[0][idx], name=data_type, line={"width":4}))

fig.update_layout(xaxis_title="Time (s)", yaxis_title="Concentration", width=1000, height=800,
                  font={"size":18},  # General font size for all elements
                  legend={"font":{"size":50}},  # Specific font size for legend
                  xaxis={"title":{"font":{"size":40}}, "tickfont":{"size":30}},  # X-axis title and tick labels
                  yaxis={"title":{"font":{"size":40}}, "tickfont":{"size":30}})   # Y-axis title and tick labels


fig.show()

fig.write_image("bevan_4-example-da-cascade-1.png", width=1200, height=800)


fig2 = go.Figure()
for data_type in ["cal12_ms.modulation_factor", "cal13_ms.modulation_factor", 
                  "kas_ms.modulation_factor", "kaf_ms.modulation_factor_g", "kaf_ms.modulation_factor_shift", 
                  "naf_ms.modulation_factor", 
                 "sk_ms.modulation_factor", ]: #all_species_data:
    idx = time >= 0.0
    data_type_str = data_type.replace("modulation_factor", "modulation")
    fig2.add_trace(go.Scatter(x=time[idx], y=all_species_data[data_type][0][0].T[0][idx], name=data_type_str, line={"width":4}))

fig2.update_layout(xaxis_title="Time (s)", yaxis_title="Modulation factor", width=1000, height=800,
                  font={"size":18},  # General font size for all elements
                  legend={"font":{"size":50}},  # Specific font size for legend
                  xaxis={"title":{"font":{"size":40}}, "tickfont":{"size":40}},  # X-axis title and tick labels
                  yaxis={"title":{"font":{"size":40}}, "tickfont":{"size":40}})   # Y-axis title and tick labels

fig2.show()
fig2.write_image("bevan_fig4_biochem.png", width=1200, height=800)

nd = None

In [13]:
from snudda.utils import SnuddaLoadSimulation

nd_on = SnuddaLoadSimulation(sim_output_neuromodulation_ON)
data_types = nd_on.list_data_types(0)
all_species_data = nd_on.get_all_data(neuron_id=0, exclude=["spikes", "voltage"])
time_on = nd_on.get_time()
voltage_on = nd_on.get_data("voltage")

Loading networks/bevan_fig4_bath_subthresh_current_SBML/simulation/output_neuromodulation_ON.hdf5


In [14]:
nd_off = SnuddaLoadSimulation(sim_output_neuromodulation_OFF)
time_off = nd_off.get_time()
voltage_off = nd_off.get_data("voltage")

Loading networks/bevan_fig4_bath_subthresh_current_SBML/simulation/output_neuromodulation_OFF.hdf5


In [15]:
voltage_off[0]

{0: array([[-0.086     , -0.086     ],
        [-0.08614558, -0.08614558],
        [-0.08623902, -0.08623902],
        ...,
        [-0.09108087, -0.09108087],
        [-0.09108087, -0.09108087],
        [-0.09108087, -0.09108087]], shape=(80001, 2))}

In [24]:
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = "iframe"  # Do not save plots in the notebook, they can get BIG


for neuron_id in voltage_on[0].keys():
    
    fig = go.Figure()
    
    sct_on = go.Scatter(x=time_on, y=voltage_on[0][neuron_id][:,0]*1e3, name="DA", opacity=0.5,  line={"width":8, "color":"red"})
    sct_off = go.Scatter(x=time_off, y=voltage_off[0][neuron_id][:,0]*1e3, name="No DA", opacity=0.5,  line={"width":8, "color":"black"})
    fig.add_traces([sct_on, sct_off])
    
    fig.update_layout(xaxis_title="Time (s)", yaxis_title="Voltage (mV)", width=1000, height=800,
                      font={"size":18},  # General font size for all elements
                      legend={"font":{"size":50}},  # Specific font size for legend
                      xaxis={"title":{"font":{"size":50}}, "tickfont":{"size":40}, "range":[0,2]},  # X-axis title and tick labels
                      yaxis={"title":{"font":{"size":50}}, "tickfont":{"size":40}, "dtick": 2})   # Y-axis title and tick labels
    fig.show()
    fig.write_image(f"bevan-figure4-{neuron_id}-volt-trace-subtresh.png", scale=2, height=800, width=1200)

In [25]:
fig.write_image(f"bevan-figure4.png", scale=2, height=800, width=1200)

## Experiment 2

Can we show persistence for 15 minutes? (Compare Figure 2)

## Experiment 3

Current injection for 3 seconds, trigger 2 (4) spikes in dSPN. After DA application (1 second, 0.5u M) spikes come earlier and at a higher frequency.

Compare Figure 4 B, C, D

## Experiment 4

Subthreshold current injection (<10 mV depolarisation). Size of depolarisation not affected by DA.

Suprathreshold current injection, with DA lower threshold, and faster spiking.

Compare Figure 5

In [ ]:
Experiment 5

Compare Figure 6